In [9]:
# Simulation of tugboat-assisted docking using SimPy.
# Models ship arrivals, tug availability, rest scheduling, and priority handling.

In [10]:
import simpy
import random
import pandas as pd

In [11]:
# Simulation parameters
NUM_TUGS = 3                  # Number of tugboats
SIM_TIME = 200                # Total hours to simulate
SHIP_ARRIVAL_MEAN = 10        # Mean time between ship arrivals
DOCKING_TIME_MEAN = 4         # Mean duration of each docking
TUG_WORK_LIMIT = 12           # Max hours a tug can work before resting
TUG_REST_TIME = 2             # Rest time after hitting work limit
PRIORITY_PROB = 0.3           # Chance a ship is high-priority (VIP)

In [12]:
# Metrics
ship_wait_times = []
tug_idle_times = {}
missed_priority_jobs = 0

In [13]:
class Tug:
    def __init__(self, env, name):
        # Each tug is a SimPy resource with capacity 1 (can assist one ship at a time)
        self.env = env
        self.name = name
        self.resource = simpy.Resource(env, capacity=1)
        self.total_work = 0
        self.idle_start = 0
        self.total_idle = 0
        self.available_at = 0  # time when tug will next be available

    def assist(self, docking_time):
        # Tug works for docking_time hours
        # Then checks if rest is needed
        self.total_work += docking_time
        yield self.env.timeout(docking_time)
        self.available_at = self.env.now

        if self.total_work >= TUG_WORK_LIMIT:
            yield self.env.timeout(TUG_REST_TIME)
            self.available_at = self.env.now
            self.total_work = 0

In [14]:
def ship_generator(env, tugs):
    ship_id = 0
    while True:
        yield env.timeout(random.expovariate(1 / SHIP_ARRIVAL_MEAN))
        ship_id += 1
        docking_time = random.expovariate(1 / DOCKING_TIME_MEAN)
        priority = 0 if random.random() < PRIORITY_PROB else 1
        env.process(handle_ship(env, f"Ship_{ship_id}", docking_time, tugs, priority))

In [15]:
def handle_ship(env, name, docking_time, tugs, priority):
    global missed_priority_jobs
    arrival_time = env.now

    # Greedy scheduling: select tug that becomes available the soonest
    chosen_tug = min(tugs, key=lambda t: max(t.available_at, env.now))

    # Request the tug, but only wait up to 5 hours
    with chosen_tug.resource.request() as req:
        result = yield req | env.timeout(5)
        # If tug accepted the request
        if req in result:
            wait_time = env.now - arrival_time
            ship_wait_times.append(wait_time)
            chosen_tug.total_idle += max(0, env.now - chosen_tug.idle_start)
            yield env.process(chosen_tug.assist(docking_time))
            chosen_tug.idle_start = env.now
            tug_idle_times[chosen_tug.name] = chosen_tug.total_idle
        # If priority ship missed due to timeout
        else:
            if priority == 0:
                missed_priority_jobs += 1

In [16]:
def run_sim():
    global ship_wait_times, tug_idle_times, missed_priority_jobs
    ship_wait_times = []
    tug_idle_times = {}
    missed_priority_jobs = 0

    env = simpy.Environment()
    tugs = [Tug(env, f"Tug_{i+1}") for i in range(NUM_TUGS)]
    env.process(ship_generator(env, tugs))
    env.run(until=SIM_TIME)

    avg_wait = sum(ship_wait_times) / len(ship_wait_times) if ship_wait_times else 0
    total_idle = sum(tug_idle_times.values())
    utilization = ((SIM_TIME * NUM_TUGS - total_idle) / (SIM_TIME * NUM_TUGS)) * 100
    return {
        "Avg. Ship Wait (h)": round(avg_wait, 2),
        "Tug Utilization (%)": round(utilization, 2),
        "Missed Priority Jobs": missed_priority_jobs
    }


In [17]:
# Run simulation for different numbers of tugs and collect performance metrics
results = []
for num_tugs in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    NUM_TUGS = num_tugs  # update global parameter
    result = run_sim()
    result["Tugs"] = num_tugs
    results.append(result)

df = pd.DataFrame(results)
print(df)

   Avg. Ship Wait (h)  Tug Utilization (%)  Missed Priority Jobs  Tugs
0                0.38                74.09                     1     2
1                0.09                79.37                     2     3
2                0.25                86.27                     1     4
3                0.35                86.58                     0     5
4                0.94                87.56                     0     6
5                0.57                90.45                     3     7
6                0.52                92.01                     1     8
7                0.49                93.12                     1     9
8                0.58                96.57                     3    10
